In [464]:
import pandas as pd
from pulp import *

In [465]:
# Define parameters to use later
num_sols = 10 # Number solutions to consider
stat_to_max = 'Poi' # Stat to maximize
constraint_stat = 'Wgt' # Stat subject to constraint
constraint_value = 25 # maximum constraint value (for weight, should be target equipment weight - weight of non-armor pieces)

In [466]:
# Read in data
armor_data = pd.read_csv('./EldenRing_Armor_Data.txt', sep='|')
armor_data.head()

,Name,Phy,VSStr,VSSla,VSPie,Mag,Fir,Lit,Hol,Imm,Rob,Foc,Vit,Poi,Wgt,SpecialEffect,HowtoAcquire,In-GameSection,ArmorType
0,Alberich's Pointed Hat,1.8,1.4,1.8,1.8,4.6,4.2,4.4,4.6,16,10,29,31,2,1.7,Strengthens thorn sorcery,The entireAlberich's Setcan be found in the ma...,2.0,Helms
1,Alberich's Pointed Hat (Altered),0.9,0.2,0.9,0.9,4.4,3.8,4.0,4.4,12,7,23,24,1,1.0,NaN,Boc the Seamstercan alter the originalAlberich...,2.0,Helms
2,Albinauric Mask,4.0,3.1,4.0,3.8,2.5,3.1,2.1,2.5,12,23,10,10,5,3.8,"+4Arcane,reduces theHPrecovery effects of theF...",Found inVolcano Manornear theGuest Hallgrace. ...,4.0,Helms
3,All-Knowing Helm,4.6,4.2,4.8,4.2,4.4,3.4,3.6,3.1,12,20,9,9,7,4.6,NaN,The entireAll-Knowing Setis dropped bySir Gide...,9.0,Helms
4,Aristocrat Hat,3.1,3.1,2.8,3.1,3.8,4.0,3.8,3.1,22,14,18,20,4,3.0,NaN,Drops fromSoldier Aristocratsthat can be found...,1.0,Helms


In [467]:
# Get list of armor types for generating pulp items
armor_types = armor_data['ArmorType'].unique().tolist()
print(armor_types)

['Helms', 'ChestArmor', 'Gauntlets', 'LegArmor']


In [468]:
# Generate necessary dictionaries
armor_dict = {} # For armor item indices
min_dict = dict(zip(armor_data.index, armor_data[constraint_stat])) # For constrained stat
stat_dict = dict(zip(armor_data.index, armor_data[stat_to_max])) # For maximized stat
for armor in armor_types: # Iterate through
    rows = armor_data.loc[armor_data['ArmorType'] == armor] # Applicable rows
    armor_dict[armor] = rows.index.tolist()  # Get items for each class

In [469]:
# Generate model
armor_prob = LpProblem("ArmorOptimizer", LpMaximize) # Model
armor_items = LpVariable.dicts('ArmorTypes', [(i, armor_type) for armor_type in armor_types
                                             for i in armor_dict[armor_type]], cat='Binary')
armor_prob += lpSum([stat_dict[i] * armor_items[i, armor] for armor in armor_types for i in armor_dict[armor]]) # Cost function
armor_prob += lpSum([min_dict[i] * armor_items[i, armor] for armor in armor_types for i in armor_dict[armor]]) <= constraint_value, "Stat constraint"
for armor in armor_types: # Only select 1 from each class
    armor_prob += lpSum([armor_items[(i, armor)] for i in armor_dict[armor]]) == 1, 'One item from %s category' % armor

In [470]:
# Other stat columns we want to track
start_col_idx = armor_data.columns.get_loc('Name') + 1
end_col_idx = armor_data.columns.get_loc('Wgt') + 1
stat_cols = armor_data.columns[start_col_idx:end_col_idx].tolist()
print(stat_cols)

['Phy', 'VSStr', 'VSSla', 'VSPie', 'Mag', 'Fir', 'Lit', 'Hol', 'Imm', 'Rob', 'Foc', 'Vit', 'Poi', 'Wgt']


In [471]:
# Iterate for multiple solutions
all_solution_sets = []
for k in range(0, num_sols): # Iterate a set number of times
    armor_prob.solve() # Solve solution
    sol_dict = {} # For storing solution info
    sol_idxs = [] # For storing selected values - add constraint later
    for armor in armor_types:
        for i in armor_dict[armor]:
            if armor_items[(i,armor)].varValue == 1:
                sol_dict[armor] = armor_data.loc[i, 'Name']
                for stat in stat_cols:
                    try: # Key exists (after first armor piece)
                        sol_dict[stat] += armor_data.loc[i, stat]
                    except: # Key doesn't exist (first armor piece)
                        sol_dict[stat] = armor_data.loc[i, stat]
                sol_idxs.append((i, armor)) # Add indexes for building constraint against repeat solution
    armor_prob += lpSum([armor_items[sol] for sol in sol_idxs]) <= 3, 'Solution %d constraint' % k
    all_solution_sets.append(sol_dict) # Append solution info

In [472]:
# Print results
result_frame = pd.DataFrame(all_solution_sets)
result_frame = result_frame[armor_types + stat_cols] # Reorganize
result_frame.head(num_sols)

,Helms,ChestArmor,Gauntlets,LegArmor,Phy,VSStr,VSSla,VSPie,Mag,Fir,Lit,Hol,Imm,Rob,Foc,Vit,Poi,Wgt
0,Omensmirk Mask,Mausoleum Knight Armor (Altered),Battlemage Manchettes,Crucible Greaves,27.3,23.1,26.9,25.6,24.2,24.5,23.2,25.4,105,113,83,85,53,24.5
1,Omensmirk Mask,Fingerprint Armor (Altered),Godskin Noble Bracelets,Crucible Greaves,27.6,24.4,25.1,24.9,22.8,25.8,19.9,23.8,103,118,76,88,53,24.3
2,Kaiden Helm,Fingerprint Armor (Altered),Battlemage Manchettes,Crucible Greaves,28.6,24.2,26.1,26.5,22.8,25.7,18.5,23.0,85,127,61,74,53,24.7
3,Champion Headband,Mausoleum Knight Armor (Altered),Godskin Noble Bracelets,Crucible Greaves,26.8,24.2,27.8,26.2,23.9,24.6,23.0,25.5,99,116,85,84,53,24.8
4,Omensmirk Mask,Mausoleum Knight Armor (Altered),Battlemage Manchettes,Scaled Greaves,26.4,23.1,26.9,25.2,24.5,25.3,24.1,25.4,109,120,85,87,53,24.8
5,Omensmirk Mask,Fingerprint Armor (Altered),Godskin Noble Bracelets,Scaled Greaves,26.7,24.4,25.1,24.5,23.1,26.6,20.8,23.8,107,125,78,90,53,24.6
6,Gilded Foot Soldier Cap,Fingerprint Armor (Altered),Godskin Noble Bracelets,Crucible Greaves,27.9,25.4,25.7,26.0,23.3,26.2,20.3,24.0,101,124,79,91,53,24.9
7,Omensmirk Mask,Fingerprint Armor,Godskin Noble Bracelets,Crucible Greaves,28.0,24.8,25.5,25.3,23.2,26.2,20.3,24.2,106,124,77,90,53,24.9
8,Marais Mask,Fingerprint Armor (Altered),Battlemage Manchettes,Tree Sentinel Greaves,27.0,23.7,24.7,24.0,24.3,29.4,20.9,24.9,103,131,91,105,53,25.0
9,Knight Helm,Fingerprint Armor (Altered),Gold Bracelets,Crucible Greaves,28.6,24.3,26.5,26.0,23.2,26.1,19.1,23.1,85,124,60,71,53,25.0
